In [5]:
import pandas as pd
import os
import numpy as np
import datetime
from joblib import Parallel, delayed
import string
import math

import gzip
import csv
#import struct

In [6]:
# import gzip
# with gzip.open(r'C:\Users\80184795\Downloads\01302019.NASDAQ_ITCH50.gz', 'rb') as f:
#     file_content = f.read()

In [7]:
import os
os.chdir(r"C:\Users\80184795\Documents\git_clones\ITCH")

In [8]:
# output_file = open("itch_sub","wb")
# output_file.write(file_content[0:10000000])
# output_file.close()

In [9]:
# # # f = open(r'C:\Users\80184795\Downloads\01302019.NASDAQ_ITCH50','rb')
# # #f = open(r'C:\Users\80184795\Documents\git_clones\ITCH\itch_sub','rb')
# f = open('/Users/yiyao/Github/ITCH/itch_sub','rb')

In [10]:
def parse_system_event_message(f, order, execute):
    global system
    data = f.read(11)
    stock_locate = int.from_bytes(data[:1], byteorder='big', signed=False)
    tracking_num = int.from_bytes(data[2:4], byteorder='big', signed=False)
    timestamp = int.from_bytes(data[4:10], byteorder='big', signed=False)
    event_code = data[10:11]
    print(stock_locate, " , ", tracking_num, " , ", timestamp, " , ", event_code)
    system[event_code] = {
        'stock_locate': stock_locate,
        'tracking_num': tracking_num,
        'timestamp': timestamp,
        'event_code': event_code}
    return timestamp
    
def parse_stock_dictionary(f, order, execute):
    data = f.read(38)
    return int.from_bytes(data[4:10], byteorder='big', signed=False)

def parse_stock_trading_action(f, order, execute):
    data = f.read(24)
    return int.from_bytes(data[4:10], byteorder='big', signed=False)

def parse_reg_sho_short_sale_price(f, order, execute):
    data = f.read(19)
    return int.from_bytes(data[4:10], byteorder='big', signed=False)
    
def parse_market_participant_position(f, order, execute):
    data = f.read(25)
    return int.from_bytes(data[4:10], byteorder='big', signed=False)
    
def parse_mwcb_decline_level_message(f, order, execute):
    data = f.read(34)
    return int.from_bytes(data[4:10], byteorder='big', signed=False)
    
def parse_mwcb_status_message(f, order, execute):
    data = f.read(11)
    return int.from_bytes(data[4:10], byteorder='big', signed=False)

def parse_ipo_quoting_period_update(f, order, execute):
    data = f.read(27)
    return int.from_bytes(data[4:10], byteorder='big', signed=False)
    
def parse_limit_up_down_auction_collar(f, order, execute):
    data = f.read(34)
    return int.from_bytes(data[4:10], byteorder='big', signed=False)

def parse_operational_halt(f, order, execute):
    data = f.read(20)
    return int.from_bytes(data[4:10], byteorder='big', signed=False)

def parse_add_order_no_mpid_attribution(f, order, execute):
    data = f.read(35)
    timestamp = int.from_bytes(data[4:10], byteorder='big', signed=False)
    reference = int.from_bytes(data[10:18], byteorder='big', signed=False)
    buysell = data[18:19]
    shares = int.from_bytes(data[19:23], byteorder='big', signed=False)
    stock = str(data[23:31], 'ascii').strip()
    price = int.from_bytes(data[31:35], byteorder='big', signed=False)
    order[reference] = {
        'timestamp': timestamp,
        'reference': reference,
        'buysell': buysell,
        'shares': shares,
        'stock': stock,
        'price': price}
    return timestamp
    
def parse_add_order_with_mpid_attribution(f, order, execute):
    data = f.read(39)
    timestamp = int.from_bytes(data[4:10], byteorder='big', signed=False)
    reference = int.from_bytes(data[10:18], byteorder='big', signed=False)
    buysell = data[18:19]
    shares = int.from_bytes(data[19:23], byteorder='big', signed=False)
    stock = str(data[23:31], 'ascii').strip()
    price = int.from_bytes(data[31:35], byteorder='big', signed=False)
    order[reference] = {
        'timestamp': timestamp,
        'reference': reference,
        'buysell': buysell,
        'shares': shares,
        'stock': stock,
        'price': price}
    return timestamp

def parse_order_executed_message(f, order, execute):
    data = f.read(30)
    timestamp = int.from_bytes(data[4:10], byteorder='big', signed=False)
    reference = int.from_bytes(data[10:18], byteorder='big', signed=False)
    executed_shares = int.from_bytes(data[18:22], byteorder='big', signed=False)
    match_number = int.from_bytes(data[22:30], byteorder='big', signed=False)
    # print(timestamp, " , ", reference, " , ", executed_shares)
    execute[match_number] = {
        'timestamp': timestamp,
        'reference': reference,
        'executed_shares': executed_shares,
        'stock': order[reference]['stock'],
        'executed_price': order[reference]['price'],
        'match_number': match_number,
        'printable': 'Y'}
    return timestamp

def parse_order_executed_with_price_message(f, order, execute):
    data = f.read(35)
    timestamp = int.from_bytes(data[4:10], byteorder='big', signed=False)
    reference = int.from_bytes(data[10:18], byteorder='big', signed=False)
    executed_shares = int.from_bytes(data[18:22], byteorder='big', signed=False)
    match_number = int.from_bytes(data[22:30], byteorder='big', signed=False)
    printable = str(data[30:31], 'ascii').strip()
    executed_price = int.from_bytes(data[31:35], byteorder='big', signed=False)
    execute[match_number] = {
        'timestamp': timestamp,
        'reference': reference,
        'executed_shares': executed_shares,
        'stock': order[reference]['stock'],
        'executed_price': executed_price,
        'match_number': match_number,
        'printable': printable}
    return timestamp
    
def parse_order_cancel_message(f, order, execute):
    data = f.read(22)
    return int.from_bytes(data[4:10], byteorder='big', signed=False)

def parse_order_delete_message(f, order, execute):
    data = f.read(18)
    return int.from_bytes(data[4:10], byteorder='big', signed=False)
    
def parse_order_replace_message(f, order, execute):
    data = f.read(34)
    timestamp = int.from_bytes(data[4:10], byteorder='big', signed=False)
    old_reference = int.from_bytes(data[10:18], byteorder='big', signed=False)
    new_reference = int.from_bytes(data[18:26], byteorder='big', signed=False)
    shares = int.from_bytes(data[26:30], byteorder='big', signed=False)
    price = int.from_bytes(data[30:34], byteorder='big', signed=False)
    order[new_reference] = {
        'timestamp': timestamp,
        'reference': new_reference,
        'buysell': order[old_reference]['buysell'],
        'shares': shares,
        'stock': order[old_reference]['stock'],
        'price': price}
    try:
        order.pop(old_reference)
    except KeyError:
        print('Key not found')
    
    return timestamp
    
def parse_non_cross_trade_message(f, order, execute):
    data = f.read(43)
    timestamp = int.from_bytes(data[4:10], byteorder='big', signed=False)
    reference = int.from_bytes(data[10:18], byteorder='big', signed=False)
    buysell = data[18:19]
    shares = int.from_bytes(data[19:23], byteorder='big', signed=False)
    stock = str(data[23:31], 'ascii').strip()
    price = int.from_bytes(data[31:35], byteorder='big', signed=False)
    match_number = int.from_bytes(data[35:43], byteorder='big', signed=False)
    execute[match_number] = {
        'timestamp': timestamp,
        'reference': reference,
        'executed_shares': shares,
        'stock': stock,
        'executed_price': price,
        'match_number': match_number,
        'printable': 'Y'} ###?? 
    return timestamp
        
def parse_cross_trade_message(f, order, execute):
    data = f.read(39)
    return int.from_bytes(data[4:10], byteorder='big', signed=False)
    
def parse_broken_trade_execution_message(f, order, execute):
    data = f.read(18)
    timestamp = int.from_bytes(data[4:10], byteorder='big', signed=False)
    match_number = int.from_bytes(data[10:18], byteorder='big', signed=False)
    execute[match_number]['executed_shares'] = 0
    return timestamp

def parse_noii_message(f, order, execute):
    data = f.read(49)
    return int.from_bytes(data[4:10], byteorder='big', signed=False)

def calculate_weighted_avg(execute, start_time, end_time):
    price_volume = {}
    for key in execute:
        timestamp = execute[key]['timestamp']
        stock = execute[key]['stock']
        volume = execute[key]['executed_shares']
        price = execute[key]['executed_price']
        if (timestamp >= start_time) and (timestamp < end_time) and (execute[key]['printable'] == 'Y'):
            if stock not in price_volume.keys():
                price_volume[stock] = {'volume': volume, 'volpri': price * volume}
            else:
                cur_volume = price_volume[stock]['volume']
                cur_volpri = price_volume[stock]['volpri']
                price_volume[stock] = {'volume': volume + cur_volume, 'volpri': price * volume + cur_volpri}
    return price_volume

handling_funcs = {
    b'S' : parse_system_event_message             ,
    b'R' : parse_stock_dictionary                 ,
    b'H' : parse_stock_trading_action             ,
    b'Y' : parse_reg_sho_short_sale_price         ,
    b'L' : parse_market_participant_position      ,
    b'V' : parse_mwcb_decline_level_message       ,
    b'W' : parse_mwcb_status_message              ,
    b'K' : parse_ipo_quoting_period_update        ,
    b'J' : parse_limit_up_down_auction_collar     ,
    b'h' : parse_operational_halt                 ,
    b'A' : parse_add_order_no_mpid_attribution    ,
    b'F' : parse_add_order_with_mpid_attribution  ,
    b'E' : parse_order_executed_message           ,
    b'C' : parse_order_executed_with_price_message,
    b'X' : parse_order_cancel_message             ,
    b'D' : parse_order_delete_message             ,
    b'U' : parse_order_replace_message            ,
    b'P' : parse_non_cross_trade_message          ,
    b'Q' : parse_cross_trade_message              ,
    b'B' : parse_broken_trade_execution_message   ,
    b'I' : parse_noii_message                     ,
}

In [12]:
hour_nanoseconds = 3600 * 1e9
t1 = datetime.datetime.now()


with open("01302019.NASDAQ_ITCH50", "rb") as f:
    avg = {}
    order = {}
    execute = {}
    system = {}
    
    start_timestamp = 0
    close_timestamp = 0
    count_hour_delta = 0
    while f.read(2):
        # Entries are separated by two zero bytes.
        timestamp = handling_funcs[f.read(1)](f, order, execute)       
        # Initialize the start_timestamp
        if start_timestamp == 0:
            try:
                start_timestamp = system[b'Q']['timestamp']
                print('Trading Starts')
            except:
                pass
        
        # General calculation
        if (timestamp - start_timestamp > hour_nanoseconds * (count_hour_delta + 1)) and (start_timestamp != 0):
            hour_avg = calculate_weighted_avg(execute, start_timestamp + count_hour_delta * hour_nanoseconds, timestamp)
                                              #start_timestamp + (count_hour_delta + 1) * hour_nanoseconds)#timestamp)
            avg[count_hour_delta] = {stock: hour_avg[stock]['volpri']/hour_avg[stock]['volume']/10000 for stock in hour_avg}
            count_hour_delta += 1
            print(count_hour_delta)
            print(timestamp)
        
        #add Trade close calculation
        if close_timestamp == 0:
            try:
                close_timestamp = system[b'M']['timestamp']
                print('Trading Ends')
                hour_avg = calculate_weighted_avg(execute, start_timestamp + count_hour_delta * hour_nanoseconds, 
                                              close_timestamp)
                avg[count_hour_delta] = {stock: hour_avg[stock]['volpri']/hour_avg[stock]['volume']/10000 for stock in hour_avg}
                break
            except:
                pass

            
t2 = datetime.datetime.now()
print(t2-t1)
# f = open('/Users/yiyao/Github/ITCH/itch_sub','rb')

0  ,  0  ,  11039687760787  ,  b'O'
0  ,  0  ,  14400000181476  ,  b'S'
0  ,  0  ,  34200000036157  ,  b'Q'
Trading Starts
1
37800000051196
2
41400000059149
3
45000000076089
4
48600000073388
5
52200000092967
6
55800000055966
0  ,  0  ,  57600000113132  ,  b'M'
Trading Ends
0:24:26.250883


In [240]:
# out_file = open('parsed_data.csv','wb')
# writer = csv.writer(out_file)
# writer.writerow(parsed_data)

In [13]:
avg

{0: {'IWM': 146.39812320001843,
  'QQQ': 163.43832434557953,
  'CME': 181.35757391447777,
  'AMZN': 1634.908231684502,
  'CROX': 28.672813791502204,
  'CRSP': 31.357357448726617,
  'CTAS': 183.14700923787527,
  'EBAY': 33.559507888662935,
  'EXEL': 23.11724919869147,
  'LABU': 44.17918413183259,
  'LABD': 28.4895319973547,
  'BSBR': 12.509123030642941,
  'LOPE': 92.46706343111657,
  'MLCO': 21.283712821364592,
  'EEM': 41.97991180966799,
  'NUVA': 48.81758471965496,
  'ARCC': 16.098441490084216,
  'NWSA': 12.595066935826603,
  'XLI': 70.65912339644508,
  'PZZA': 42.42226567028235,
  'BUD': 74.46745423953004,
  'SRPT': 119.2259227296588,
  'ABT': 69.91469855803669,
  'STX': 43.13395553257497,
  'TER': 36.55006212765957,
  'RWT': 15.85307113611036,
  'AZN': 36.08299453505191,
  'WBA': 71.36789697100093,
  'CNHI': 10.018195533736455,
  'CTSH': 68.84759774125341,
  'EXPE': 117.64618806783767,
  'FGEN': 53.702681863531865,
  'BWA': 40.04936676631718,
  'AU': 13.710311636328983,
  'LNT': 42.

In [14]:
len(avg)

7

In [ ]:
df = pd.DataFrame.from_dict(avg[0], orient='index').rename(columns={0: "VWAP"}).sort_index()

In [ ]:
import pandas as pd

# Create some Pandas dataframes from some data.
writer = pd.ExcelWriter('output.xlsx', engine='xlsxwriter')
for i in avg:
    df = pd.DataFrame.from_dict(avg[i], orient='index').rename(columns={0: "VWAP"}).sort_index()
    df.to_excel(writer, sheet_name="string{0}".format(i))

writer.save()

In [34]:
# use excute as input, test count_hour_delta = 0
# price_volume should be blah
# avg[0] should be balh
start_time = start_timestamp
end_time = start_timestamp + hour_nanoseconds
blah = calculate_weighted_avg(execute, start_time, end_time)

In [36]:
blah

{'IWM': {'volume': 695146, 'volpri': 1017680697500},
 'QQQ': {'volume': 1485169, 'volpri': 2427335327300},
 'CME': {'volume': 48502, 'volpri': 87962050500},
 'AMZN': {'volume': 400890, 'volpri': 6554183610000},
 'CROX': {'volume': 21535, 'volpri': 6174690450},
 'CRSP': {'volume': 13311, 'volpri': 4173977850},
 'CTAS': {'volume': 8660, 'volpri': 15860531000},
 'EBAY': {'volume': 1107412, 'volpri': 371642017500},
 'EXEL': {'volume': 121052, 'volpri': 27983892500},
 'LABU': {'volume': 77917, 'volpri': 34423094900},
 'LABD': {'volume': 65021, 'volpri': 18524178600},
 'BSBR': {'volume': 39226, 'volpri': 4906828600},
 'LOPE': {'volume': 6511, 'volpri': 6020530500},
 'MLCO': {'volume': 84289, 'volpri': 17939828700},
 'EEM': {'volume': 1684425, 'volpri': 707120129500},
 'NUVA': {'volume': 9738, 'volpri': 4753856400},
 'ARCC': {'volume': 29448, 'volpri': 4740669050},
 'NWSA': {'volume': 51766, 'volpri': 6519962350},
 'XLI': {'volume': 773126, 'volpri': 546284054350},
 'PZZA': {'volume': 24154, 

In [37]:
{stock: blah[stock]['volpri']/blah[stock]['volume']/10000 for stock in blah}

{'IWM': 146.39812320001843,
 'QQQ': 163.43832434557953,
 'CME': 181.35757391447777,
 'AMZN': 1634.908231684502,
 'CROX': 28.672813791502204,
 'CRSP': 31.357357448726617,
 'CTAS': 183.14700923787527,
 'EBAY': 33.559507888662935,
 'EXEL': 23.11724919869147,
 'LABU': 44.17918413183259,
 'LABD': 28.4895319973547,
 'BSBR': 12.509123030642941,
 'LOPE': 92.46706343111657,
 'MLCO': 21.283712821364592,
 'EEM': 41.97991180966799,
 'NUVA': 48.81758471965496,
 'ARCC': 16.098441490084216,
 'NWSA': 12.595066935826603,
 'XLI': 70.65912339644508,
 'PZZA': 42.42226567028235,
 'BUD': 74.46745423953004,
 'SRPT': 119.2259227296588,
 'ABT': 69.91469855803669,
 'STX': 43.13395553257497,
 'TER': 36.55006212765957,
 'RWT': 15.85307113611036,
 'AZN': 36.08299453505191,
 'WBA': 71.36789697100093,
 'CNHI': 10.018195533736455,
 'CTSH': 68.84759774125341,
 'EXPE': 117.64618806783767,
 'FGEN': 53.702681863531865,
 'BWA': 40.04936676631718,
 'AU': 13.710311636328983,
 'LNT': 42.87856626613717,
 'SNPS': 90.7175606038

,VWAP
A,73.762976
AA,28.844582
AABA,66.677925
AAC,2.251391
AAL,36.415227
AAME,2.750000
AAN,48.751657
AAOI,16.651404
AAON,35.675902
AAP,158.355987
